<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Topic-Modeling" data-toc-modified-id="Topic-Modeling-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Topic Modeling</a></span><ul class="toc-item"><li><span><a href="#Getting-Started" data-toc-modified-id="Getting-Started-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Getting Started</a></span></li><li><span><a href="#Wiki-Example" data-toc-modified-id="Wiki-Example-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Wiki Example</a></span></li></ul></li><li><span><a href="#Reference" data-toc-modified-id="Reference-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Reference</a></span></li></ul></div>

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim import corpora
from gensim.models import LdaModel
from operator import itemgetter
from nltk.corpus import stopwords
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS


## Topic Modeling

Topic modeling is a technique for taking some unstructured text and automatically extracting its common themes, it is a great way to get a bird's eye view on a large text collection.

The major feature distinguishing topic model from other clustering methods is the notion of mixed membership. A lot of clustering models have assumed that each data point belongs to a single cluster. **K-means** determines membership simply by shortest distance to the cluster center, and **Gaussian Mixture Models** assumes that each data point is drawn from one of their component mixture distributions. In many cases, though, it is more realistic to think of data as genuinely belonging to more than one cluster or category. For example, if we have a model for text data that includes both "Politics" and "World News" categories, then an article about a recent meeting of the United Nations should have membership in both categories rather than being forced into just one.

Topic model assumes that a topic is a probability distribution over the vocabulary. For example, if we were to create three topics for the Harry Potter series of books manually, we might come up with something like this:

- (Muggle topic) 50% "muggle", 25% "Dursley", 10% "Privet", 5% "mudblood"...
- (Voldemort topic) 65% "Voldemort", 12% "death", 10% "Horcrux", 5% "snake"...
- (Harry Potter topic) 42% "Harry Potter", 15% "scar", 7% "Quidditch", 7% "Gryffindor"...

The way we interpret these words is, it is the probability of those words appearing in the topic to the left. So, an example interpretation of the output above will be: There is 42% chance that the word "Harry Potter" came from the Harry Potter topic. Note that the vocabulary probability will sum up to 1 for every topic, but often times, words that have lower weights will be truncated from the output.

In the same way, we can represent individual documents as a probability distribution over topics. For example, Chapter 1 of Harry Potter book 1 introduces the Dursley family and has Dumbledore discuss Harry’s parent’s death. If we take this chapter to be a single document, it could be broken up into topics like this: 40% Muggle topic, 30% Voldemort topic, and the remaining 30% is the Harry topic.

Of course, we don't want to extract the topics and document probabilities by hand like this. We want the machine to do it automatically using our unlabelled text collection as the only input. Because there is no document labeling nor human annotations, topic modeling is an example of an unsupervised machine learning technique.

### Getting Started

We'll now start exploring one popular algorithm for doing topic model, namely Latent Dirichlet Allocation. Latent Dirichlet Allocation (LDA) requires documents to be represented as a bag of words (for the gensim library, some of the API calls will shorten it to bow, hence we'll use the two interchangeably). This representation ignores word ordering in the document but retains information on how many times each word appears.

We'll first play around a toy corpus of 11 documents to familiarize ourselves with the gensim API. In the toy corpus presented, there are 5 documents that are river related and 6 finance related. The interesting part is that the corpus contains the word "bank" which could mean "a financial institution" or "a river bank". A good topic model should be able to tell the difference between these two meanings based on context.

In [6]:
# each nested list represents the word of a document
texts = [['bank','river','shore','water'],
        ['river','water','flow','fast','tree'],
        ['bank','water','fall','flow'],
        ['bank','bank','water','rain','river'],
        ['river','water','mud','tree'],
        ['money','transaction','bank','finance'],
        ['bank','borrow','money'], 
        ['bank','finance'],
        ['finance','money','sell','bank'],
        ['borrow','sell'],
        ['bank','loan','sell']]

# build the dictionary and convert the documents 
# to bag of words (bow) representation using the dictionary
texts_dictionary = corpora.Dictionary(texts)
texts_corpus = [texts_dictionary.doc2bow(text) for text in texts]

The following code chunk trains the LDA model using our corpus and dictionary. We set the number of topics to be 2, and expect to see one related to river banks, and one has to do with financial banks.

In [7]:
# train the model
# the more iteration, the more stable the model
# becomes, but of course takes longer to train
np.random.seed(431)
texts_model = LdaModel( 
    texts_corpus, 
    id2word = texts_dictionary, 
    num_topics = 2,
    passes = 5,
    iterations = 50)

In [8]:
# we can pass the num_words argument to limit the listed 
# number of most probable words 
texts_model.show_topics()

[(0,
  u'0.214*"bank" + 0.131*"sell" + 0.130*"finance" + 0.127*"money" + 0.092*"borrow" + 0.056*"loan" + 0.054*"transaction" + 0.028*"shore" + 0.025*"river" + 0.025*"water"'),
 (1,
  u'0.181*"water" + 0.147*"bank" + 0.147*"river" + 0.084*"flow" + 0.083*"tree" + 0.050*"mud" + 0.050*"rain" + 0.050*"fall" + 0.050*"fast" + 0.042*"shore"')]

After using the show_topics method from the model, it will output the most probable words that appear in each topic. For the gensim library, the default printing behavior is to print a linear combination of the top words sorted in decreasing order of the probability of the word appearing in that topic. Thus words that appear towards the left are the ones that are more indicative of the topic.

We see our LDA model has given us a pretty intuitive result. Bank is the most influential word in both the topics and other words help define what kind of bank we are talking about. We can also use the function get_term_topics and get_document_topics to further evaluate our result. get_term_topics returns the odds of that particular word belonging to a particular topic. A few examples:

In [9]:
print(texts_model.get_term_topics('water'))
print(texts_model.get_term_topics('bank'))

[(1, 0.16697583)]
[(0, 0.19870603), (1, 0.13300718)]


Since the word bank is likely to be in both the topics, the values returned are also very similar.

The get_document_topics method outputs the topic distribution of the document. Apart from this, it also let us know the topic distribution for each word in the document. Let us test this with two different documents which have the word bank in it, one in the finance context and one in the river context.

In [10]:
# before we can infer topic distributions on new, unseen documents
# we need to convert it to bag of words format first
bow_water = ['bank', 'water', 'bank']
bow = texts_model.id2word.doc2bow(bow_water) 
doc_topics, word_topics, phi_values = texts_model.get_document_topics(
    bow, per_word_topics = True)

# note that doc_topics equivalent to simply calling model[bow]
print('document topics: ', doc_topics)
print()

for word_id, topic in word_topics:
    # access the word with the word id
    print(texts_model.id2word[word_id], topic)

print()

for word_id, topic in phi_values:
    print(texts_model.id2word[word_id], topic)

('document topics: ', [(0, 0.26637727), (1, 0.73362273)])
()
(u'bank', [1, 0])
(u'water', [1, 0])
()
(u'bank', [(0, 0.54996675), (1, 1.4500314)])
(u'water', [(0, 0.014795795), (1, 0.98520327)])


What do all these output mean?

doc_topics shows that this document has a higher probability of belonging to topic 1.
When per_word_topics is set as True, it also returns a word_topics. This variable captures the word id followed by a list sorted with the most likely topic id. From the output, it means both our word bank and water is more likely to be in topic 1 than topic 0.
We could also list out the phi_values. Compared with word_topics, it adds the information of the probability of the word belonging to a particular topic. Note that it is scaled by feature length (the word bank appeared 2 times, hence the value of the probability will add up to 2).
let's now do the same thing with a second document, bow_finance.

In [11]:
bow_finance = ['bank', 'finance']
bow = texts_model.id2word.doc2bow(bow_finance)  # convert to bag of words format first
doc_topics, word_topics, phi_values = texts_model.get_document_topics(
    bow, per_word_topics = True)
word_topics

[(0, [0, 1]), (10, [0])]

Since the word bank is now used in a financial context, the most probable topic for the word immedietly swaps to being topic 0. We've seen quite clearly that based on the context, the most likely topic associated with a word can change. This differs from our previous method, get_term_topics, where it is a 'static' topic distribution.

But note that, each word in a document is only given one topic distribution, meaning it can't tell the difference if one word that have different meanings occured in the same document (every 'bank' word will have the same distribution).

bow = texts_model.id2word.doc2bow(['the', 'bank', 'by', 'the', 'river', 'bank'])
doc_topics, word_topics, phi_values = texts_model.get_document_topics(
    bow, per_word_topics = True)
word_topics

### Wiki Example

## Reference

- [Blog: Gensim LDA: Tips and Tricks](http://miningthedetails.com/blog/python/lda/GensimLDA/)
- [Notebook: Pre-processing and training LDA](http://nbviewer.jupyter.org/github/RaRe-Technologies/gensim/blob/develop/docs/notebooks/lda_training_tips.ipynb)
- [Coursera Washington Clustering & Retrieval](https://www.coursera.org/learn/ml-clustering-and-retrieval)
- [gensim documentation: Corpora and Vector Spaces](http://nbviewer.jupyter.org/github/RaRe-Technologies/gensim/blob/develop/docs/notebooks/Corpora_and_Vector_Spaces.ipynb)
- [gensim documentation: New Term Topics Methods and Document Coloring](http://nbviewer.jupyter.org/github/RaRe-Technologies/gensim/blob/develop/docs/notebooks/topic_methods.ipynb)
- [Quora: Why is the performance improved by using TFIDF instead of bag-of-words in LDA clustering?](https://www.quora.com/Why-is-the-performance-improved-by-using-TFIDF-instead-of-bag-of-words-in-LDA-clustering)